In [2]:
from IPython.display import HTML, display
import os
import cv2
import numpy as np
from ipywidgets import widgets, Layout, interact
import pandas as pd

In [3]:
hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
    <input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code.">
</form>''')

In [4]:
hide_me

path = widgets.Text()
print("Path: ")
display(path)

def handle_path(sender):
    global PATH
    PATH = path.value
    print("Path updated !")

Path: 


Text(value='')

In [5]:
def read_files(directory, limit):
    global PATH
    files=[]
    for file in os.listdir(directory):
        if limit <=0:
            break
        image = cv2.imread(os.path.join(directory, file))
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        files.append(image)
        limit-=1
    return files

images = read_files('data/images/', 10)
expected_results = read_files('data/manual1/', 10)
masks = read_files('data/mask/', 10)

In [83]:
def process_image(img):
    #normalizacja histogramu kolorów
    img = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    img[:,:,0] = cv2.equalizeHist(img[:,:,0])
    res = cv2.cvtColor(img, cv2.COLOR_YUV2BGR)
    #rozmycie
    res = cv2.medianBlur(res,7)
    return res


img = images[0]
filtered = process_image(img)

cv2.imshow("original", img)
cv2.imshow("filtered", filtered)
cv2.waitKey(0)
cv2.destroyAllWindows()


#images=[process_image(img) for img in images]

In [29]:
def split_image(img,exp, width, height):
    img_width = img.shape[0]
    img_height = img.shape[1]
    tiles = [img[x:x+width,y:y+height] for x in range(0,img_width,width) for y in range(0,img_height,height)]
    values =[exp[x,y] for x in range(width//2,img_width,width) for y in range(height//2,img_height,height)]
    return tiles, values


In [30]:
def get_data(img):
    variance=np.var(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    moments = cv2.moments(gray)
    huMoments = cv2.HuMoments(moments)
    result = [variance, moments, huMoments]
    return result


img=images[0]
fragments, values=split_image(img,expected_results[0], 5, 5)
data=[]
for frag in fragments:
    data.append(get_data(frag))

print(data[0][0])

0.3811555555555555


In [8]:
print(data[0][2])

[[ 1.81375000e-01]
 [ 2.80189063e-03]
 [ 1.00642788e-03]
 [ 4.53053828e-05]
 [ 3.54680353e-09]
 [-8.34042599e-07]
 [-9.00059724e-09]]


In [9]:
expected_results[0][0]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=uint8)

In [40]:
def createDataFrame(data):
    final_frame = pd.DataFrame()
    idx = ['hu0','hu1','hu2','hu3','hu4','hu5','hu6']
    for example in data:
        df1 = pd.DataFrame.from_dict(data = example[1], orient='index')
        df2 = pd.DataFrame.from_dict(data = {"variance":example[0]}, orient ='index') 
        df3 = pd.DataFrame(example[2],index=idx)
        frame = df2.append(df1.append(df3))
        frame = frame.T
        #print("FRAME ###########")
        #print(frame)
        final_frame = final_frame.append(frame, ignore_index=True)
        #print("FINAL FRAME ###############")
        #print(final_frame)
    return final_frame

def addDecision(frame, dec):
    for i in range(0,len(dec)):
        if sum(dec[i]) == 0:
            dec[i] = 0
        else:
            dec[i] = 1
    frame['vessel'] = dec
    return frame

In [41]:
df_final = createDataFrame(data)

KeyboardInterrupt: 

In [79]:
df_final

,variance,m00,m10,m01,m20,m11,m02,m30,m21,m12,...,nu21,nu12,nu03,hu0,hu1,hu2,hu3,hu4,hu5,hu6
0,0.416233,45.0,171.0,154.0,839.0,609.0,702.0,4575.0,3093.0,2865.0,...,0.003009,0.002540,0.002943,0.179841,0.000602,0.000167,0.000037,-2.579236e-09,-5.805786e-07,-1.315888e-09
1,0.437473,42.0,172.0,169.0,932.0,685.0,877.0,5464.0,3571.0,3669.0,...,-0.010591,0.011772,-0.007441,0.240700,0.000366,0.003541,0.000377,-4.359854e-07,-6.837591e-06,3.307327e-09
2,0.218641,18.0,60.0,58.0,302.0,188.0,284.0,1686.0,940.0,1036.0,...,0.001778,0.089991,-0.053061,0.614540,0.001312,0.076558,0.010641,-6.704543e-05,3.834329e-04,-2.962390e-04


In [ ]:
df_final = addDecision(df_final, values)

In [84]:
images[0].shape

(2336, 3504, 3)

In [10]:
masks[0].shape

(2336, 3504, 3)

In [23]:
expected_results[0].shape

(2336, 3504, 3)

In [19]:
expected_results[0][1444][1784]

array([0, 0, 0], dtype=uint8)

In [ ]:
#decision data - how to do it XD

In [35]:
len(values)

327367

In [38]:
sum([255,255,255])

765